 # **The Battle of the Neighborhoods - Week 1**



# 1. Introduction # 

**Business problem and who would be interested in this project**: 

Pizza its your favorite food and you live in USA but you dont know about "pizza places". So the problem we want to solve is to analyze high density of Pizza places around USA. First, we want to know the density of Pizza stores' in the most important cities from USA. Then, we will find the best place looking for many Pizza stores.


# 2. Data # 

**Description of the data that will be used to solve the problem and the source of the data:** 

- API: Foursquare 


- Objective: collect data about locations of Pizza stores.


- Cities to analyze: New York,NY, San Francisco, CA, Jersey City, NJ, Boston, MA and Chicago,IL. 

# 3. Methodology # 

**Main component of the report:**

- Import Libraries
- Connection with Foursquare
- We defined different maps with concentrations of pizza stores for each City.
- After the first results, we measure the density using mean location and take the average of the distance of the venues to the mean coordinates.



# **The Battle of the Neighborhoods - Week 2** 



# 4. Code # 

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
CLIENT_ID = 'WW4AAKC1T55UQRGELZYBG2JOZP4Y4JCX4MTIEC2JNFQ0PJIJ' # your Foursquare ID
CLIENT_SECRET = 'EPC1W40OJ3AVFRNX0GOO1AWT1ZJBILLMCHD03P52N0ELKUPU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: WW4AAKC1T55UQRGELZYBG2JOZP4Y4JCX4MTIEC2JNFQ0PJIJ


In [8]:
# type your answer here
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") # PIZZA PLACE CATEGORY ID
    results[city] = requests.get(url).json()

In [9]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [10]:

maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in New York, NY =  258
Showing Top 100
Total number of pizza places in Chicago, IL =  219
Showing Top 100
Total number of pizza places in San Francisco, CA =  168
Showing Top 100
Total number of pizza places in Jersey City, NJ =  126
Showing Top 100
Total number of pizza places in Boston, MA =  183
Showing Top 100


In [11]:
maps[cities[0]]

In [12]:
maps[cities[1]]

In [13]:
maps[cities[2]]

In [14]:
maps[cities[3]]

In [15]:
maps[cities[4]]

We can see that New York and Jersey are the most dense cities with Pizza places. Lets measure that density using mean location of the pizza places which should be near to most of them if they are really dense or far if not. Then we will take the average of the distance of the venues to the mean coordinates.

In [16]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.0225057926387654
Chicago, IL
Mean Distance from Mean coordinates
0.06108761995659042
San Francisco, CA
Mean Distance from Mean coordinates
0.02900595707851886
Jersey City, NJ
Mean Distance from Mean coordinates
0.02958265550428421
Boston, MA
Mean Distance from Mean coordinates
0.037284357369393666


# 5. Results 

**Discussion of the results. Observations you noted and any recommendations you can make based on the results:**



Another observation is that there is one really far away Pizza store which would possible increase its score to be beaten by New York So let's try to remove it and calculate it again

In [17]:

city = 'Jersey City, NJ'
venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 

print(city)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

Jersey City, NJ
Mean Distance from Mean coordinates
0.021791097611226277


Jersey its the best option!!!

# 6. Conclusion 


After an analysis of the maps of these large cities we see that both the statistical techniques and the analysis after their use are extremely useful. Thanks to Foursquare we were able to identify that Jersey is the great pizza place!